### Objetivo:
Aqui, vamos limpar os dados e incluir algumas colunas que serão úteis 

In [155]:
import pandas as pd
import pre_processsing as pp
%reload_ext autoreload
%autoreload 2

### Carregar os dados

In [230]:
dtype = {'sku': 'object'}
columns = ['sku', 'descricao', 'gross_price', 'price', 'pagamento', 'date', 'day', 'hour']

df_bw = pd.read_csv('../../data/beleza_na_web.csv', decimal=',', dtype=dtype, encoding='latin-1', usecols=columns) 
df_sep = pd.read_csv('../../data/sephora.csv', decimal='.', dtype=dtype, encoding='latin-1', usecols=columns)
df_ep = pd.read_csv('../../data/epoca.csv', decimal=',', dtype=dtype, encoding='latin-1', usecols=columns) 

df_bw['price'] = df_bw['price'].apply(lambda x:float(x.strip().replace(".","").replace(",",".")))

### Vamos focar num dia apenas

In [231]:
to_filter = "day == '05/06/20'"

df_bw = df_bw.query(to_filter)
df_sep = df_sep.query(to_filter)
df_ep = df_ep.query(to_filter)

In [232]:
[print('df_bw'), df_bw.info(), 
 print('---'),
 print('df_sep'), df_sep.info(),
 print('df_ep'), df_ep.info()]

df_bw
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8028 entries, 14940 to 22967
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          8028 non-null   object 
 1   descricao    8028 non-null   object 
 2   gross_price  3684 non-null   float64
 3   price        8028 non-null   float64
 4   pagamento    6147 non-null   object 
 5   date         8028 non-null   object 
 6   day          8028 non-null   object 
 7   hour         8028 non-null   object 
dtypes: float64(2), object(6)
memory usage: 564.5+ KB
---
df_sep
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8025 entries, 14309 to 22333
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          8025 non-null   object 
 1   descricao    8025 non-null   object 
 2   gross_price  7926 non-null   float64
 3   price        7926 non-null   float64
 4   pagamento    7926 non-null   o

[None, None, None, None, None, None, None]

### Quantidade de registros únicos

In [233]:
['unicos:', {'df_bw': df_bw['sku'].nunique(), 
          'df_sep': df_sep['sku'].nunique(),
            'df_ep': df_ep['sku'].nunique()}]

['unicos:', {'df_bw': 420, 'df_sep': 356, 'df_ep': 333}]

### Acrescentar colunas

In [242]:
# Coluna com o player
df_bw = df_bw.assign(player='belezanaweb')
df_ep = df_ep.assign(player='epoca')
df_sep = df_sep.assign(player='sephora')

# Juntar as 3 bases
df_cons = pd.concat([df_bw,df_ep,df_sep]).reindex()

# Trazar apenas produto com preço maior que zero e ordenar 
df_cons = df_cons.sort_values(['player', 'sku','date']).query("price > 0")

# Coluna booleana com True se houve alteração de preço em relação a hora anterior
df_cons = df_cons.assign(change_price=df_cons.groupby(['player','sku'])['price'].diff().fillna(0).apply(lambda x: True if x != 0 else False))

# Demais colunas
df_cons = (df_cons.assign(date=pd.to_datetime(df_cons['date'])))
df_cons = (df_cons.assign(gross_price=df_cons['gross_price'].fillna(0)))
df_cons = df_cons.query("price > 0")
df_cons = df_cons.assign(gross_price=df_cons.apply(lambda x: x['price'] if x['gross_price'] == 0 else x['gross_price'], axis=1))
df_cons = df_cons.assign(desconto=df_cons.apply(lambda x: True if x['price']<x['gross_price'] else False, axis=1))
df_cons = df_cons.assign(perc_des=df_cons.apply(lambda x: (x['price']/x['gross_price']-1)*100 if x['desconto'] is True else 0, axis=1).round(2))
df_cons = df_cons.assign(valor_des=df_cons.apply(lambda x: (x['price']-x['gross_price']) if x['desconto'] is True else 0, axis=1).round(2))

### Amostra dos dados

In [ ]:
df_cons.sample(20)

### Salvar dados em csv

In [339]:
pd.DataFrame.to_csv(df_cons, '../../data/dados_consolidados.csv', index=False)